In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# df = pd.read_csv('/kaggle/input/high_diamond_ranked_10min.csv')
df = pd.read_csv('./sample_data/high_diamond_ranked_10min.csv')

# 캐글 데이터톤 데이터 셋 EDA, 시각화
https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min



*   높은 랭크(다이아몬드~마스터)에서의 통계입니다.
*   팀당 19개의 피쳐( 총 38개)가 있습니다.
* blueWins 열이 목표값이고 값이 1이면 파란색 팀이 이겼다는 의미이고 그렇지 않으면 0입니다.

* 10분 전에 게임 내의 어떤 결정이 이기는데 가장 관련 있는지 알아봅니다. 

<br>

---

<Br>

1.   처음에 BlueSide에만 집중을한다.
2.   데이터 세트를 탐색하여 패턴과 상관 관계를 확인한다.
3.   탐색한 후 다시 feature를 재조합 한다.
4.   디시젼 트리를 시각화를 하여 상관 관계를 다시 한번 확인한다.
5. 어떤 feature가 게임을 이기는데 가장 관련 되어있는지 시각화 한다.


In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# 결측치 유무
df.isnull().sum()

In [ ]:
# df.blueWins.map()

In [ ]:
df['whoWins'] = df.blueWins.map({0:"REDwins", 1:"BlueWins"})
palette = ['RED','BLUE']
plt.figure(figsize=(4,3),dpi=200)
sns.countplot(data=df, x='whoWins', palette=palette)

In [ ]:
df.whoWins.value_counts()

 이 부분에 확인 할수 있는 부분은 레드팀과 블루팀의 결과가 거의 정확히 절반이다. 당연히 블루가 이기면 레드가 졌을테니 블루팀만 보면 레드팀의 결과도 확인할 수있으니 블루팀 feature만 보도록 한다.

In [ ]:
# 블루 사이드 관련 컬럼으로만 데이터 만들어놓기
blue_side_columns=[]
for col in df:
  if 'blue' in col:
    blue_side_columns.append(col)

blue_data = df[blue_side_columns]
blue_data.head()

In [ ]:
describe_blue = blue_data.describe()
describe_blue

In [ ]:
# 블루팀 진영 이긴 특성과의 다른 데이터의 상관관계 분포도
plt.figure(figsize=(12,10))
sns.heatmap(blue_data.drop('blueWins',axis=1).corr(),
            cmap='vlag', annot=True, fmt='.2f', vmin=-1)

In [ ]:
blue_data_cleaned = blue_data.copy()

In [ ]:
# 킬 어시 의 값 모아주기 
blue_data_cleaned['KA'] = blue_data_cleaned['blueAssists']+blue_data_cleaned['blueKills']
# blue_data_cleaned['KDA'] = blue_data_cleaned['blueAssists']+blue_data_cleaned['blueKills']/blue_data_cleaned['blueDeaths']

# blue_data_cleaned['KDA'].fillna(0)
# blue_data_cleaned['KDA']


blue_data_cleaned = blue_data_cleaned.drop(['blueAssists','blueKills'],axis=1)


In [ ]:
blue_data_cleaned.info()

# 킬 어시를 모아준 이유 


1.   롤 게임 내에서는 어시스트 란 킬을 했을때 도와준 값을 어시스트라고 한다.
2.   그러므로 킬 어시를 따로두고 봐야할 이유가 없다고 생각해서
하나의 feature로 모아줘도 무방하다고 생각했습니다.



In [ ]:
blue_data_cleaned.head()

In [ ]:
Gold = []
for column in list(blue_data_cleaned.columns):
  if 'Gold' in column:
    Gold.append(column)
Gold

In [ ]:
blue_data_cleaned = blue_data_cleaned.drop(Gold,axis=1)
blue_data_cleaned.columns

# 골드의 값을 삭제한 이유


1.   리그오브레전드 에서 골드를 얻는 상황은
* 1초당 1골드, 
* 킬,어시를 한 상황일때, 
* 밑에있는 EliteMonters(Dragon, Herald) 사냥 유무,
* 미니언 처치 시 일 때 골드를 얻는다.


---


2. 위 3개의 features 설명

* 게임 초반 10내의 블루팀 전체 골드(blueTotalGold) 
*   블루팀 기준 레드팀과의 골드차이(blueGoldDiff) 
*   블루팀 기준 10분 얻은 골드량(blueGoldPerMin) (blueTotalGold)와 같은 값입니다.


---



그러므로 위의 3개의 컬럼을 삭제하고 
* 킬 어시(KA)
* 블루팀 총 미니언 처치 수(blueTotalMinionsKilled)
* 블루팀 용 처치 수(blueDragon)
* 블루팀 전령 처치 수(blueHerald)

으로 봐도 무방 하다고 생각했습니다.




In [ ]:
blue_data_cleaned = blue_data_cleaned.drop(['blueEliteMonsters'],axis=1)

In [ ]:
Experience =[]
for column in list(blue_data_cleaned.columns):
  if 'Experience' in column:
    Experience.append(column)
Experience
blue_data_cleaned = blue_data_cleaned.drop(Experience,axis=1)

In [ ]:
blue_data_cleaned = blue_data_cleaned.drop(['blueCSPerMin'],axis=1)

# blueEliteMonsters , blueCSPerMin 삭제 이유


1.   blueEliteMonsters 삭제는 위의 이유와 동일
2.   blueCSPerMin 값은 게임 초반 10분 내의 블루팀 미니언 처치량 값이라 blueGoldPerMin 위의 이유와 동일



In [ ]:
# 총 컬럼 과 총 컬럼 수
blue_data_cleaned.columns , blue_data_cleaned.columns.shape

In [ ]:
plt.figure(figsize=(12, 10))
# sns.heatmap(blue_data_cleaned.drop("blueWins",axis=1).corr(),cmap='vlag', annot=True, fmt='.2f', vmin=-1)
sns.heatmap(blue_data_cleaned.corr(),cmap='vlag', annot=True, fmt='.2f', vmin=-1)

# 특이치 확인 및 삭제

In [ ]:
# 특이치 확인
blue_no_outliers = blue_data_cleaned.copy()

sns.displot(blue_no_outliers['blueWardsPlaced'],kind="ecdf")

In [ ]:
blue_no_outliers = blue_no_outliers.loc[blue_no_outliers['blueWardsPlaced']
                                        <= np.quantile(blue_no_outliers['blueWardsPlaced']
                                                      ,q=0.99)]
print('삭제 {}'.format(df.shape[0]-blue_no_outliers.shape[0]))

In [ ]:
sns.displot(blue_no_outliers['blueWardsDestroyed'],kind='ecdf')

In [ ]:
blue_no_outliers = blue_no_outliers.loc[blue_no_outliers['blueWardsDestroyed'] 
                                        <= np.quantile(blue_no_outliers['blueWardsDestroyed']
                                                       ,q=0.99)]
print('삭제 {}'.format(df.shape[0]-blue_no_outliers.shape[0]))

In [ ]:
sns.displot(blue_no_outliers['blueDeaths'],kind='ecdf')

In [ ]:
blue_data_scaled = blue_no_outliers.copy()

blue_target = blue_no_outliers['blueWins'] #target
blue_features = blue_no_outliers.drop(['blueWins'],axis=1) # features

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
scaler.fit(blue_features)

blue_features_scaled = scaler.transform(blue_features)

In [ ]:
blue_features_scaled.mean(axis=0)
blue_features_scaled.std(axis=0)

In [ ]:
# Train Test Split
x_train, x_test, y_train, y_test = train_test_split(blue_features_scaled,blue_target, random_state=42, test_size=0.10)

logReg = LogisticRegression()
logReg.fit(blue_features_scaled,blue_target)

logReg.score(blue_features_scaled,blue_target)

In [ ]:
blue_weights = pd.DataFrame()
blue_weights['Features'] = blue_features.columns
blue_weights['Weights'] = logReg.coef_.reshape(11,1)

In [ ]:
blue_weights.sort_values(by='Weights',ascending=False)

In [ ]:
blue_data_cleaned.columns[1:]

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics

using_cols = blue_data_cleaned.columns[1:]

X = blue_data_cleaned[using_cols]# columns
y = blue_data_cleaned['blueWins'] # target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# Decision Tree  모델 학습
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(random_state=2,
                             max_depth=5)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus

dot_data = StringIO()
export_graphviz(clf, 
                out_file=dot_data,  
                filled=True, 
                rounded=True,
                special_characters=True, 
                feature_names=using_cols, 
                class_names=['redWins','blueWins']
               )
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('row_df.png')
Image(graph.create_png(), retina=True)

In [ ]:
plt.figure(figsize=(15,15))
corrmat = df.corr()
corrmat = np.tril(corrmat)
corrmat[corrmat==0] = None
corrmat = corrmat.round(1)
labels = df.select_dtypes(include='number').columns.values
f, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(corrmat, annot=True, vmax=0.8,vmin=-0.8, cmap='seismic_r', xticklabels=labels,yticklabels=labels, cbar=False)
plt.legend('')

plt.show()